In [1]:
from test import EigenData
import numpy as np
import scipy.io
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
%matplotlib inline

In [2]:
data = EigenData()
eigenworms = data.get_eigenworms('EigenWorms.mat')

footage = data.get_footage('20150814-All-PNAS2011-DataStitched .mat')
newdata = torch.rand(33600,5)
for k in footage.keys():
    print(k, footage[k].shape)
    arr1=footage[k]

    arr2=arr1[:5,]

    r = data.reconstruct(footage[k])
    print('-->', r.shape)
    dataset1 = arr2.transpose()

    newdata = np.concatenate((newdata,dataset1),axis=0)
newdata = newdata[33600:,]
print(newdata.shape)

b (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
c (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
d (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
e (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
f (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
g (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
h (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
i (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
j (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
k (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
l (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
m (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
(403200, 5)


In [3]:
first_5_eigenworms = eigenworms[0:5,:]
eig_worm_0 = eigenworms[0,:]
eig_worm_1 = eigenworms[1,:]
eig_worm_2 = eigenworms[2,:]
eig_worm_3 = eigenworms[3,:]
eig_worm_4 = eigenworms[4,:]

In [4]:
data1=newdata[~np.isnan(newdata).any(axis=1)]

data1 = data1.astype('float32')
print(data1.shape)
print(len(data1))

(386422, 5)
386422


In [5]:
def create_dataset(data,n_predictions,n_next):
    
   
    dim = data.shape[1]
    train_X, train_Y = [], []
    for i in range(data.shape[0]-n_predictions-n_next-1):
        a = data[i:(i+n_predictions),:]
        train_X.append(a)
        tempb = data[(i+n_predictions):(i+n_predictions+n_next),:]
        b = []
        for j in range(len(tempb)):
            for k in range(dim):
                b.append(tempb[j,k])
        train_Y.append(b)
    train_X = np.array(train_X,dtype='float64')
    train_Y = np.array(train_Y,dtype='float64')

    return train_X, train_Y

In [6]:
X, Y = create_dataset(data1,3,6)
print(X.shape)
print(Y.shape)

(386412, 3, 5)
(386412, 30)


In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
data1 = scaler.fit_transform(data1)
Y = scaler.fit_transform(Y)

In [8]:
data_X, data_Y = create_dataset(data1,3,6)

print(data_X.shape)
print(data_Y.shape)

(386412, 3, 5)
(386412, 30)


In [9]:
train_size = int(len(data_X) * 0.8)
test_size = len(data_X) - train_size
train_X = data_X[:train_size]
train_Y = Y[:train_size]
test_X = data_X[train_size:]
test_Y = Y[train_size:]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(309129, 3, 5)
(309129, 30)
(77283, 3, 5)
(77283, 30)


In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input,LSTM,Dropout,Dense


timesteps = 3
# create and fit the LSTM network

model = keras.Sequential()
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(30,activation='linear'))
model.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(train_X, train_Y, epochs=100, batch_size=512, verbose=2, validation_data=(test_X, test_Y))

Epoch 1/100
604/604 - 22s - loss: 0.0134 - root_mean_squared_error: 0.1159 - val_loss: 0.0061 - val_root_mean_squared_error: 0.0782
Epoch 2/100
604/604 - 17s - loss: 0.0061 - root_mean_squared_error: 0.0779 - val_loss: 0.0039 - val_root_mean_squared_error: 0.0626
Epoch 3/100
604/604 - 17s - loss: 0.0041 - root_mean_squared_error: 0.0639 - val_loss: 0.0020 - val_root_mean_squared_error: 0.0448
Epoch 4/100
604/604 - 17s - loss: 0.0028 - root_mean_squared_error: 0.0527 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0384
Epoch 5/100
604/604 - 16s - loss: 0.0022 - root_mean_squared_error: 0.0472 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0386
Epoch 6/100
604/604 - 17s - loss: 0.0019 - root_mean_squared_error: 0.0441 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0324
Epoch 7/100
604/604 - 16s - loss: 0.0018 - root_mean_squared_error: 0.0418 - val_loss: 9.6346e-04 - val_root_mean_squared_error: 0.0310
Epoch 8/100
604/604 - 16s - loss: 0.0016 - root_mean_squared_error: 0.04

Epoch 61/100
604/604 - 16s - loss: 9.2561e-04 - root_mean_squared_error: 0.0304 - val_loss: 5.4480e-04 - val_root_mean_squared_error: 0.0233
Epoch 62/100
604/604 - 16s - loss: 9.2128e-04 - root_mean_squared_error: 0.0304 - val_loss: 5.1748e-04 - val_root_mean_squared_error: 0.0227
Epoch 63/100
604/604 - 16s - loss: 9.1584e-04 - root_mean_squared_error: 0.0303 - val_loss: 5.7349e-04 - val_root_mean_squared_error: 0.0239
Epoch 64/100
604/604 - 16s - loss: 9.1327e-04 - root_mean_squared_error: 0.0302 - val_loss: 5.0766e-04 - val_root_mean_squared_error: 0.0225
Epoch 65/100
604/604 - 15s - loss: 9.0790e-04 - root_mean_squared_error: 0.0301 - val_loss: 4.8502e-04 - val_root_mean_squared_error: 0.0220
Epoch 66/100
604/604 - 15s - loss: 9.0345e-04 - root_mean_squared_error: 0.0301 - val_loss: 4.8388e-04 - val_root_mean_squared_error: 0.0220
Epoch 67/100
604/604 - 16s - loss: 8.9996e-04 - root_mean_squared_error: 0.0300 - val_loss: 4.6455e-04 - val_root_mean_squared_error: 0.0216
Epoch 68/100


In [11]:
testPredict = model.predict(test_X)

In [12]:
newtestPredict = scaler.inverse_transform(testPredict)
newtest_Y = scaler.inverse_transform(test_Y)

In [13]:
testScore = sqrt(mean_squared_error(newtest_Y, newtestPredict))
print('Test Score: %.5f RMSE' % (testScore))

Test Score: 0.58729 RMSE
